In [2]:
import pandas as pd

In [27]:
# Code permettant de produire le fichier csv contenant les données d'évaluation à enlever (considéré comme des silences)

import data
import torch
import numpy as np


energy_2s = {'mix_name': [], 'voices': [], 'eval_frame': [], 'energy': []}

el_rossinyol = data.CSD(song_name='El Rossinyol', example_length=64000, allowed_voices='satb',
                        return_name=True, n_sources=2, singer_nb=[2], random_mixes=False,
                        f0_from_mix=True, cunet_original=False)

locus_iste = data.CSD(song_name='Locus Iste', example_length=64000, allowed_voices='satb',
                        return_name=True, n_sources=2, singer_nb=[3], random_mixes=False,
                        f0_from_mix=True, cunet_original=False)

nino_dios = data.CSD(song_name='Nino Dios', example_length=64000, allowed_voices='satb',
                    return_name=True, n_sources=2, singer_nb=[4], random_mixes=False,
                    f0_from_mix=True, cunet_original=False)

test_set = torch.utils.data.ConcatDataset([el_rossinyol, locus_iste, nino_dios])

device = 'cuda' if torch.cuda.is_available() else 'cpu'

energy_total = []
for idx in range(len(test_set)):
    # Load batch of 4 1-second frames
    d = test_set[idx]

    mix = d[0].to(device)
    f0_hz = d[1].to(device)
    target_sources = d[2].to(device)
    name = d[3]
    voices = d[4]

    mix = mix.unsqueeze(0)
    target_sources = target_sources.unsqueeze(0)
    f0_hz = f0_hz[None, :, :]

    batch_size, n_samples, n_sources = target_sources.shape

    n_fft_metrics = 512
    overlap_metrics = 0.5
    

    target_sources = target_sources.transpose(1, 2)  # [batch_size, n_sources, n_samples]
    target_sources = target_sources.reshape((batch_size * n_sources, n_samples))
    
    n_source, n_samples = target_sources.shape
    n_eval_frames = int(np.ceil(n_samples / 16000))
    
    energy_batch = []
    for source in range(n_source):
        for n in range(n_eval_frames):
            energy_2s['mix_name'].append(name)
            energy_2s['voices'].append(voices[source])
            energy_2s['eval_frame'].append(n)
            energy_2s['energy'].append(torch.sum(torch.square(target_sources[source, n*16000: (n+1)*16000])).cpu().numpy() / 2.8767)

In [ ]:
# Code permettant de produire le fichier csv contenant les données d'évaluation à enlever (considéré comme des silences)
# Utilise le fichier "energy_snippets_4s.pandas"
# nécessite de changer data.py, la classe CSD afin de renvoyer audio_start_time et song_name.

import data
import torch
import numpy as np

energy_snippet_4s = pd.read_pickle('./Datasets/ChoralSingingDataset/energy_snippets_4s.pandas')

energy_2s = {'mix_name': [], 'voices': [], 'eval_frame': [], 'energy': []}

el_rossinyol = data.CSD(song_name='El Rossinyol', example_length=64000, allowed_voices='satb',
                        return_name=True, n_sources=2, singer_nb=[2], random_mixes=False,
                        f0_from_mix=True, cunet_original=False)

locus_iste = data.CSD(song_name='Locus Iste', example_length=64000, allowed_voices='satb',
                        return_name=True, n_sources=2, singer_nb=[3], random_mixes=False,
                        f0_from_mix=True, cunet_original=False)

nino_dios = data.CSD(song_name='Nino Dios', example_length=64000, allowed_voices='satb',
                    return_name=True, n_sources=2, singer_nb=[4], random_mixes=False,
                    f0_from_mix=True, cunet_original=False)

test_set = torch.utils.data.ConcatDataset([el_rossinyol, locus_iste, nino_dios])

device = 'cuda' if torch.cuda.is_available() else 'cpu'

energy_total = []
for idx in range(len(test_set)):
    # Load batch of 4 1-second frames
    d = test_set[idx]

    mix = d[0].to(device)
    f0_hz = d[1].to(device)
    target_sources = d[2].to(device)
    name = d[3]
    audio_start_time = d[4]
    song_name = d[5]
    voices = d[6]

    mix = mix.unsqueeze(0)
    target_sources = target_sources.unsqueeze(0)
    f0_hz = f0_hz[None, :, :]

    batch_size, n_samples, n_sources = target_sources.shape

    n_fft_metrics = 512
    overlap_metrics = 0.5
    

    target_sources = target_sources.transpose(1, 2)  # [batch_size, n_sources, n_samples]
    target_sources = target_sources.reshape((batch_size * n_sources, n_samples))
    
    n_source, n_samples = target_sources.shape
    n_eval_frames = int(np.ceil(n_samples / 16000))
    
    energy_batch = []
    for source in range(n_source):
        for n in range(n_eval_frames):
            energy_2s['mix_name'].append(name)
            energy_2s['voices'].append(voices[source])
            energy_2s['eval_frame'].append(n)
            # energy_2s['energy'].append(torch.sum(torch.square(target_sources[source, n*16000: (n+1)*16000])).cpu().numpy() / 2.8767)
            energy_snippet_4s['energy'][ (energy_snippet_4s['eval_frame'] == n) &  (energy_snippet_4s['song_name'] == song_name) & (energy_snippet_4s['start_time'] == audio_start_time) & (energy_snippet_4s['voice'] == voices[n_source])]

In [28]:
energy_2s = pd.DataFrame(energy_2s)
energy_2s

,mix_name,voices,eval_frame,energy
0,el_rossinyol_Sno_208_At_2-06_3.984,s,0,69.938705
1,el_rossinyol_Sno_208_At_2-06_3.984,s,1,91.915268
2,el_rossinyol_Sno_208_At_2-06_3.984,s,2,55.305416
3,el_rossinyol_Sno_208_At_2-06_3.984,s,3,190.038635
4,el_rossinyol_Sno_208_At_2-06_3.984,a,0,81.653374
...,...,...,...,...
4891,nino_dios_t4-06-2_Bos_404_95.984,t,3,205.545517
4892,nino_dios_t4-06-2_Bos_404_95.984,b,0,88.376427
4893,nino_dios_t4-06-2_Bos_404_95.984,b,1,81.060524
4894,nino_dios_t4-06-2_Bos_404_95.984,b,2,47.207012


In [30]:
energy_2s.to_pickle("./Datasets/ChoralSingingDataset/" + '/energy_snippets_2s.pandas')

### Eval Model with MIR metrics

In [ ]:
df_pierre = pd.read_pickle('./umss/evaluation/unsupervised_4s_satb_bcbq_mf0_1_again_masking/eval_results_mf0_CSD_cuda/all_results.pandas')

In [ ]:
df_pierre = pd.read_pickle('./umss/evaluation/unsupervised_4s_satb_bcbq_mf0_1_again_masking/eval_results_mf0_CSD_cuda/all_results.pandas')

# drop des signaux avec une énergie trop faible
energy_snippet = pd.read_pickle('./umss/Datasets/ChoralSingingDataset/energy_snippets.pandas')
energy_to_drop = []
for i, energy in enumerate(energy_snippet["energy"]):
    if energy < 10:
        for j in range(1):
            energy_to_drop.append(i + j * 1632)
            
df_result = df_pierre.drop(energy_to_drop)

In [ ]:
means = df_result.mean(axis=0, skipna=True, numeric_only=True)
medians = df_result.median(axis=0, skipna=True, numeric_only=True)
stds = df_result.std(axis=0, skipna=True, numeric_only=True)

In [ ]:
print('SI-SDR [mean, median, std]:', means['SI-SDR'], medians['SI-SDR'], stds['SI-SDR'])
print('sp_SNR [mean, median, std]:', means['sp_SNR'], medians['sp_SNR'], stds['sp_SNR'])
print('sp_SI-SNR [mean, median, std]:', means['sp_SI-SNR'], medians['sp_SI-SNR'], stds['sp_SI-SNR'])
print('mel_cep_dist [mean, median, std]:', means['mel_cep_dist'], medians['mel_cep_dist'], stds['mel_cep_dist'])

### Eval Robustness of Model

In [3]:
df = pd.read_pickle('/home/ids/chouteau/umss/evaluation/2s_CuestaInTrained_Fix_HCQT_Torch_bc1song_again_new_assignement_masking/eval_results_mf0_CSD_cuda/all_results.pandas')

In [4]:
means = df.mean(axis=0, skipna=True, numeric_only=True)
print(means)

eval_seed        2.000000
eval_frame       1.509642
sp_SNR          15.232910
sp_SI-SNR       15.488824
SI-SDR          13.502447
mel_cep_dist    13.971754
dtype: float64


### Energy_test

In [ ]:
energy_snippet = pd.read_pickle('/home/pchouteau/umss/umss/energy_test_snippets.pandas')

energy_to_drop = []
for i, energy in enumerate(df["energy"]):
    if energy < 10:
        for j in range(5):
            energy_to_drop.append(i + j * 1632)
            
df_final = df_pierre.drop(energy_to_drop)

In [ ]:
means = df_final.mean(axis=0, skipna=True, numeric_only=True)
medians = df_final.median(axis=0, skipna=True, numeric_only=True)
stds = df_final.std(axis=0, skipna=True, numeric_only=True)

print('SI-SDR [mean, median, std]:', means['SI-SDR'], medians['SI-SDR'], stds['SI-SDR'])
print('sp_SNR [mean, median, std]:', means['sp_SNR'], medians['sp_SNR'], stds['sp_SNR'])
print('sp_SI-SNR [mean, median, std]:', means['sp_SI-SNR'], medians['sp_SI-SNR'], stds['sp_SI-SNR'])
print('mel_cep_dist [mean, median, std]:', means['mel_cep_dist'], medians['mel_cep_dist'], stds['mel_cep_dist'])